# Evaluate simulation likelihood given data

In [ ]:
# Packages

import numpy as np

from scipy import integrate, stats
from scipy.special import expit, binom

import pandas as pd
import xlrd

import copy
import warnings

from datetime import datetime
import random
import string
import os
import shutil
import sys
import cloudpickle
import dask
import distributed
from dask.distributed import Client
import itertools

import pymc3

import inspect
from collections import OrderedDict

import dask.dataframe as dd


import functools

# Plotly 
import cufflinks as cf
import plotly
from plotly.offline import iplot as plt
from plotly import graph_objs as plt_type
from plotly import graph_objs as go
plotly.offline.init_notebook_mode(connected=True)

cf.go_offline()

# Helper functions

Distributed dask workers seem very fiddly at importing non-standard modules, so it's better to just copy-paste the functions into this notebook sadly

In [ ]:
# To get relatve age-related risks, we have to first re-group into our basic age groups, 
# then devide by total population (here's a non-well-defined subset of UK, so absolute values wont be used, only relative)
import numpy as np
import inspect
from collections import OrderedDict

def regroup_by_age(inp, fromAgeSplits, toAgeSplits, maxAge=100., maxAgeWeight = 5.):
    fromAgeSplits = np.concatenate([np.array([0]), fromAgeSplits, np.array([maxAge])]) # Add a zero at beginning for calculations
    toAgeSplits = np.concatenate([np.array([0]), toAgeSplits, np.array([maxAge])]) # Add inf at end for calculations
    def getOverlap(a, b):
        return max(0, min(a[1], b[1]) - max(a[0], b[0]))
    out = np.zeros((len(toAgeSplits)-1,)+inp.shape[1:])
    for from_ind in range(1, len(fromAgeSplits)):
        # Redistribute to the new bins by calculating how many years in from_ind-1:from_ind falls into each output bin
        cur_out_distribution = (
        [getOverlap(toAgeSplits[cur_to_ind-1:cur_to_ind+1],fromAgeSplits[from_ind-1:from_ind+1])  for cur_to_ind in range(1, len(toAgeSplits))]
        )
        
        if cur_out_distribution[-1] > 0:
            cur_out_distribution[-1] = maxAgeWeight # Define the relative number of ages if we have to distribute between second to last and last age groups

        cur_out_distribution = cur_out_distribution/np.sum(cur_out_distribution)
        
        for to_ind in range(len(out)):
            out[to_ind] += cur_out_distribution[to_ind] * inp[from_ind-1]
            
    return out


# PARAMETER DICTIONARIES AND TABLES
# -----------------------------------------------------------------------------------------


def build_paramDict(cur_func):
    """
    This function iterates through all inputs of a function, 
    and saves the default argument names and values into a dictionary.
    
    If any of the default arguments are functions themselves, then recursively (depth-first) adds an extra field to
    the dictionary, named <funcName + "_params">, that contains its inputs and arguments.
    
    The output of this function can then be passed as a "kwargs" object to the highest level function, 
    which will then pass the parameter values to the lower dictionary levels appropriately
    """
    
    paramDict = OrderedDict()
    
    allArgs = inspect.getfullargspec(cur_func)
    
    # Check if there are any default parameters, if no, just return empty dict
    if allArgs.defaults is None:
        return paramDict
    
    
    for argname, argval in zip(allArgs.args[-len(allArgs.defaults):], allArgs.defaults):
        # Save the default argument
        paramDict[argname] = argval
        # If the default argument is a function, inspect it for further 
        
        if callable(argval):
            # print(argname)
            paramDict[argname+"_params"] = build_paramDict(argval)

    return paramDict




# Do a mapping between dictionary and parameter table row (for convenient use)

# Flatten the dictionary into a table with a single row (but many headers):
def paramDict_toTable(paramDict):
    paramTable = pd.DataFrame()
    def paramDictRecurseIter(cur_table, cur_dict, preString):
        # Iterate through the dictionary to find all keys not ending in "_params", 
        # and add them to the table with name <preString + key>
        # 
        # If the key doesn end in "_params", then append the key to preString, in call this function on the value (that is a dict)
        for key, value in cur_dict.items():
            if key.endswith("_params"):
                paramDictRecurseIter(cur_table, value, preString+key+"_")
            else:
                paramTable[preString+key] = [value]
                
        # For the rare case where we want to keep an empty dictionary, the above for cycle doesn't keep it
        if len(cur_dict)==0:
            paramTable[preString] = [OrderedDict()]
                
        return cur_table
    
    return paramDictRecurseIter(paramTable, paramDict, preString="")

# Example dict -> table
# paramTable_default = paramDict_toTable(paramDict_default)
    

def paramTable_toDict(paramTable, defaultDict=None):
    # enable to pass a default dict (if paramTable is incomplete), in which we'll just add / overwrite the values
    paramDict = defaultDict if defaultDict is not None else OrderedDict() 
    def placeArgInDictRecurse(argName, argVal, cur_dict):
        # Find all "_params_" in the argName, and for each step more and more down in the dictionary
        strloc = argName.find("_params_")
        if strloc == -1:
            # We're at the correct level of dictionary
            cur_dict[argName] = argVal
            return cur_dict
        else:
            # step to the next level of dictionary
            nextKey = argName[:strloc+len("_params_")-1]
            nextArgName = argName[strloc+len("_params_"):]
            if not nextKey in cur_dict:
                cur_dict[nextKey] = OrderedDict()
            placeArgInDictRecurse(nextArgName, argVal, cur_dict[nextKey])
            
        return cur_dict
            
    for key in paramTable.columns:
        paramDict = placeArgInDictRecurse(key, paramTable.at[0,key], paramDict)
        
    return paramDict

# Example table -> dict 
# paramDict_new = paramTable_toDict(paramTable_default)

## Load datasets

In [ ]:
CONST_DATA_CUTOFF_DATE = "20200414"

### NHS England deaths dataset

In [ ]:
# NHS daily deaths report (about 24 hours behind)
# TODO manually update link and column numbers (maybe not consistent across days, cannot yet automate)
df_UK_NHS_daily_COVID_deaths = pd.read_excel(
    "https://www.england.nhs.uk/statistics/wp-content/uploads/sites/2/2020/04/COVID-19-total-announced-deaths-22-April-2020.xlsx",
    sheet_name = "COVID19 total deaths by age",
    index_col=0,
    usecols = "B,E:AX",
    skip_rows = range(17),
    nrows = 22
).iloc[14:].transpose().set_index("Age group").rename_axis(index = "Date", columns = "AgeGroup")

df_UK_NHS_daily_COVID_deaths.index = pd.to_datetime(df_UK_NHS_daily_COVID_deaths.index, format="%Y-%m-%d")

df_UK_NHS_daily_COVID_deaths = df_UK_NHS_daily_COVID_deaths.drop(df_UK_NHS_daily_COVID_deaths.columns[:2], axis=1)

df_UK_NHS_daily_COVID_deaths

# Ignore very recent unreliable data points
df_UK_NHS_daily_COVID_deaths = df_UK_NHS_daily_COVID_deaths.loc[
            df_UK_NHS_daily_COVID_deaths.index <= CONST_DATA_CUTOFF_DATE]

df_UK_NHS_daily_COVID_deaths

### NHS England CHESS - COVID hospitalisations - dataset

In [ ]:
# Load the aggregate data (ask @SebastianVollmer for aggregation details!)
df_CHESS = pd.read_csv("/mnt/efs/data/CHESS_Aggregate20200417.csv").drop(0)

# Clean the dates and make them index
# The "1899-12-30" is simply total, ignore that.
# The 2020-09-03, 2020-10-03, 2020-11-03, 2020-12-03 are parsed wrong and are march 09-12 supposedly.
# The data collection is only officially started across england on 09 March, the February dates seem empty, delete.
# Rest are ok

df_CHESS.index = pd.to_datetime(df_CHESS["DateOfAdmission"].values,format="%d-%m-%Y")

# Ignore too old and too recent data points
df_CHESS = df_CHESS.sort_index().drop("DateOfAdmission", axis=1).query('20200309 <= index <= ' + CONST_DATA_CUTOFF_DATE)

df_CHESS


In [ ]:
df_CHESS.columns

In [ ]:
# Get the hospitalised people who tested positive for COVID, using cumsum (TODO: for now assuming they're all still in hospital)
df_CHESS_newCOVID = df_CHESS.loc[:,df_CHESS.columns.str.startswith("AllAdmittedPatientsWithNewLabConfirmedCOVID19")]

df_CHESS_newCOVID.sort_index()

## Define likelihoods

In [ ]:
# Load an example simulation:
simExample = np.load("/mnt/efs/results/run_20200408T195337/outTensor_20200408T195337_slr7ep10hy0q9iyr3k36.npy")
simExample_newOnly = np.load("/mnt/efs/results/run_20200408T195337/outTensor_20200408T195337_slr7ep10hy0q9iyr3k36_newOnly.npy")

In [ ]:
# We expect 
def joinDataAndSimCurves(
    df_curves, # a pandas dataframe with dates as index, and each column is a curve
    simCurves, # curves x time np array
    simStartDate, # curves start dates
    simCurvesNames = None,
    fulljoin = False # if true, then one keeps all dates in the simulation, instead of just the ones in the date 
    ):
    
    out_df = copy.deepcopy(df_curves)
    
    simCurveIndex = pd.date_range(start=simStartDate, freq='D', periods=simCurves.shape[1])
    
    if simCurvesNames is None:
        simCurvesNames = ["simCurve_{}".format(i) for i in range(simCurves.shape[0])]
    
    join_type = "outer" if fulljoin else "left"
    
    for i, curve in enumerate(simCurves):
        out_df = out_df.join(
            pd.DataFrame(
                index = simCurveIndex,
                data = simCurves[i],
                columns=[simCurvesNames[i]]
            ),
            how = join_type
        )
    
    return out_df

In [ ]:
# Dummy lik function to plot curves without data, get correct projection via functools.partial(..., projFunc = ...)
def likFunc_dummyForPlotting(
    sim, 
    simStartDate, 
    df = None,
    sumAges = True,
    outputDataframe = False, # If true, outputs the data-curves and simulated curves instead of likelihood, for plotting
    fulljoin = False, # if true, then one keeps all dates in the simulation, instead of just the ones in the date
    projFunc = lambda sim: np.diff(np.sum(sim[:,-1,:,:,:],axis=(1,2)),-1) # pass a lambda function to create a simCurve, this example does daily new deaths
    ):
    
    
    simCurves = projFunc(sim)
    if sumAges:
        simCurves = simCurves.sum(0, keepdims=True)
    
    if df is None:
        df = pd.DataFrame(index= pd.date_range(start=simStartDate, freq='D', periods=simCurves.shape[1])) # an empty pandas dataframe with dates as index
    else:
        if sumAges:
            df = pd.DataFrame(df.sum(1))
            
    # Join the two dataframes to align in time
    df_full = joinDataAndSimCurves(
        df_curves = df,
        simCurves = simCurves, # curves x time np array
        simStartDate = simStartDate, # curves start dates
        fulljoin = fulljoin
    )
    
    # If true, outputs the data-curves and simulated curves instead of likelihood, for plotting
    if outputDataframe:
        return df_full
    
    return None
    
    

In [ ]:
def getNegBinomParams(mu, alpha):
    """ 
    From https://stats.stackexchange.com/questions/260580/negative-binomial-distribution-with-python-scipy-stats
    Convert mean/dispersion parameterization of a negative binomial to the ones scipy supports

    Parameters
    ----------
    mu : float 
       Mean of NB distribution.
    alpha : float
       Overdispersion parameter used for variance calculation.

    See https://en.wikipedia.org/wiki/Negative_binomial_distribution#Alternative_formulations
    """
    var = mu + alpha * mu ** 2
    p = (var - mu) / var
    r = mu ** 2 / (var - mu)
    return r, p

def NegBinom_logpmf(a, m, x):
    binom_vec = np.array([binom(x1 + a - 1, x1) for x1 in x])
    logpmf = np.log(binom_vec) + a * np.log(a / (m + a))  + x * np.log((m / (m + a)))
    return logpmf

#### Deaths

In [ ]:
def likFunc_deaths(
    sim, # use newOnly for deaths by day needed here
    simStartDate, 
    df,
    sumAges = True,
    outputDataframe = False, # If true, outputs the data-curves and simulated curves instead of likelihood, for plotting
    fulljoin = False # if true, then one keeps all dates in the simulation, instead of just the ones in the date
    ):

        
    # Get deaths by day in simulation in hospitals for people with positive tests
    deaths_Sim_byAge = np.sum(sim[:,-1,2,:,:],axis=(1))
    
    
    if sumAges:
        deaths_Sim = np.sum(deaths_Sim_byAge,axis=0, keepdims=True) 
        deaths_data = pd.DataFrame(df.sum(1))
    else:
        # Change the grouping of ages to be same as dataset
        deaths_Sim_byAge_regroup = regroup_by_age(
            deaths_Sim_byAge, 
            fromAgeSplits = np.arange(10,80+1,10), 
            toAgeSplits = np.arange(20,80+1,20)
        )
        
        deaths_Sim = deaths_Sim_byAge_regroup
        deaths_data = df
        
    
    
    # Join the two dataframes to align in time
    df_full = joinDataAndSimCurves(
        df_curves = deaths_data, # a pandas dataframe with dates as index, and each column is a curve
        simCurves = deaths_Sim, # curves x time np array
        simStartDate = simStartDate, # curves start dates
        fulljoin = fulljoin
    )
    
    # If true, outputs the data-curves and simulated curves instead of likelihood, for plotting
    if outputDataframe:
        return df_full
    
    # We assume the order of columns in data and in simCurves are the same!
    #return df_full
    return np.nansum(
        NegBinom_logpmf(2., 
                        # Select all simCurve columns and reshape to a single vector
                        m = 1e-8+np.reshape(df_full.loc[:,df_full.columns.str.startswith("simCurve_")==True].values,-1), 
                        # Select all data columns and reshape to a single vector
                        x = np.reshape(df_full.loc[:,(df_full.columns.str.startswith("simCurve_")==True)==False].values,-1)
                       )
    )
 

In [ ]:
likFunc_deaths(
    sim = simExample, 
    simStartDate = '2020-03-12',
    df = copy.deepcopy(df_UK_NHS_daily_COVID_deaths),
    sumAges=True
)

In [ ]:
def likFunc_newHospPositive(
    sim, # Here we'll make sure to pass the "_newOnly" tensor!
    simStartDate, 
    df,
    sumAges = True,
    outputDataframe = False, # If true, outputs the data-curves and simulated curves instead of likelihood, for plotting
    fulljoin = False # if true, then one keeps all dates in the simulation, instead of just the ones in the date
    ):
    """
    Get the number of hospitalised people testing positive each day.
    This fits well with "policyFunc_testing_symptomaticOnly" being active, which prioratises testing hospitalised people
    As per 09 April this is a very reasonable assumption
    """
    
    # Calculate the simulation curves of hospitalised people getting positive tests each day
    # TODO - run the simulation with actual test numbers each day, would make fits a LOT better.
    # Take into account the number of positive tested people who leave the hospital and add that as well 
    # (as they were also replaced by new people testing positive in the hospital!)
    
    # Change in hospital and testing positive
    newHospPositives_sim = np.sum(sim[:,:,2,1,:], axis=(1,))
    
    if sumAges:
        hospPos_Sim = np.sum(newHospPositives_sim,axis=0, keepdims=True) 
        hospPos_data = pd.DataFrame(df.sum(1))
    else:
        
        # Change the grouping of ages to be same as dataset
        
        hospPos_Sim = regroup_by_age(
            newHospPositives_sim, 
            fromAgeSplits = np.arange(10,80+1,10), 
            toAgeSplits = np.concatenate([np.array([1,5,15,25]),np.arange(45,85+1,10)])
        )
        
        hospPos_data = df
        
        
    # Join the two dataframes to align in time
    df_full = joinDataAndSimCurves(
        df_curves = hospPos_data, # a pandas dataframe with dates as index, and each column is a curve
        simCurves = hospPos_Sim, # curves x time np array
        simStartDate = simStartDate, # curves start dates
        fulljoin = fulljoin
    )
    
    # If true, outputs the data-curves and simulated curves instead of likelihood, for plotting
    if outputDataframe:
        return df_full
    
    
    # We assume the order of columns in data and in simCurves are the same!
    #return df_full
    return np.nansum(
        NegBinom_logpmf(2., 
                        # Select all simCurve columns and reshape to a single vector
                        m = 1e-8+np.reshape(df_full.loc[:,df_full.columns.str.startswith("simCurve_")==True].values,-1), 
                        # Select all data columns and reshape to a single vector
                        x = np.reshape(df_full.loc[:,(df_full.columns.str.startswith("simCurve_")==True)==False].values,-1)
                       )
    )
    

In [ ]:
likFunc_newHospPositive(
    sim = simExample_newOnly, 
    simStartDate = '2020-03-12',
    df = copy.deepcopy(
        df_CHESS_newCOVID
    ),
    sumAges=True
)

## Load simulations and evaluate

In [ ]:
#loadDir = "/mnt/efs/results/run_20200411T134739/" # <- large base ensemble of random simulations
#loadDir = "/mnt/efs/results/run_20200414T210254/" # <- GP based ensemble
loadDir = "/mnt/efs/results/run_20200421T002117/" # <- NEW - GP based ensemble


In [ ]:
i = 0
while True:
    print("Loading {}".format(i))
    try:
        if i == 0:
            paramTable_loaded_dd = dd.from_pandas(pd.read_hdf(loadDir + 'paramTable_part0', key="paramTable"),chunksize=1000)
        else:
            paramTable_loaded_dd = paramTable_loaded_dd.append(pd.read_hdf(loadDir + 'paramTable_part{}'.format(i), key="paramTable"))
        i += 1
    except:
        break
        
len(paramTable_loaded_dd)

In [ ]:
(paramTable_loaded_dd[["likelihood_0", "likelihood_1", "likelihood_2", "likelihood_3"]]
 .compute()
 .rename(columns={
     "likelihood_0": "likDailyTotalDeaths", 
     "likelihood_1": "likDailyTotalDeaths_byAgeGroups", 
     "likelihood_2": "likDailyNewPosTestsHospital",
     "likelihood_3": "likDailyNewPosTestsHospital_byAgeGroups"
 })
).iplot(subplots=True, subplot_titles=True, kind="histogram")

In [ ]:
# Filter rows based on likelihoods (see above histograms for value choices)
paramTable_goodOnly = (paramTable_loaded_dd
    #.query("likelihood_1 > -650")
    .query("likelihood_0 > -260 & likelihood_2 > -280")
    #.query("likelihood_total > -3000")
    #.query("likelihood_0 > -200 & likelihood_2 > -250")
    #.query("likelihood_0 > -140 & likelihood_1 > -350 & likelihood_2 > -200 & likelihood_3 > -1300")
 #.query("likelihood_1 > -400 & likelihood_3 > -2100")
 .compute()
                      )

print(len(paramTable_goodOnly))
paramTable_goodOnly.sort_values("likelihood_total").tail(30).loc[:,"likelihood_0":"likelihood_total"]

## Helper funcs for organising data and simulations for plotting

In [ ]:
def tryLoad(
    fn,
    shape = (9,8,4,3,100,),
    fill = 0.
    ):
    try:
        return np.load(fn)
    except:
        return fill*np.ones(shape)

In [ ]:
def joinSimulations(
    loadDir,
    paramTable,
    simRowIndices, # from paramTables filled with bestStartTime column
    likFunc,
    simSuffix,
    df, # data
    paramTable_startTimeColname = "bestStartTime",
    sumAges = True
    ):
    
    # Get the individual data frames that contain the appropriate projection(s) of the simulation to match the data
    all_df_full = [
        likFunc(
            sim = tryLoad(loadDir + paramTable.at[ind,"out_fname"][:-4] + simSuffix + ".npy"), 
            simStartDate = paramTable.at[ind, paramTable_startTimeColname],
            df = df,
            sumAges=sumAges,
            outputDataframe=True,
            fulljoin=True
        )
        
        for ind in simRowIndices
    ]
    
    
    df_data_cols = all_df_full[0].loc[:,(all_df_full[0].columns.str.startswith("simCurve_")==True)==False]
    
    origSimColNames = all_df_full[0].loc[:,(all_df_full[0].columns.str.startswith("simCurve_")==True)].columns
   
    
    df_out = df_data_cols.join(
        # Get only the simCurve columns from the dataframes and rename them to include the simulation row index
        other = [
            cur_df.loc[:,cur_df.columns.str.startswith("simCurve_")==True].rename(
                columns = {s: "row_" + str(rowInd) + "_" + s for s in origSimColNames}
            )
            
            for cur_df, rowInd in zip(all_df_full, simRowIndices)
        ],
        
        how = "outer"
    )
    
    return df_out
    
    
    

## Plot simulations against data

In [ ]:
def plot(Y, X=None, now = True, **kwargs):
    """
    plots a matrix Y (n x num_lines)
    with optionally x spacing
    """
    plots = list()
    
    if len(Y.shape)==1:
        Y = Y[:,np.newaxis]
    
    if X is None:
        X = np.arange(Y.shape[0])
    
    for num_line in range(Y.shape[1]):
        plots.append(
            plt_type.Scatter(
                x = X[:,num_line] if len(X.shape)==2 else X,
                y = Y[:,num_line],
                **kwargs
            )
        )
        
    if now:
        plt(plots)
    else:
        return plots

In [ ]:
# Plot new deaths per day
tmp = joinSimulations(
    loadDir = loadDir,
    paramTable=paramTable_goodOnly,
    simRowIndices = paramTable_goodOnly.sort_values("likelihood_0").tail(20).index,
    paramTable_startTimeColname = "realStartDate",
    likFunc = likFunc_deaths,
    simSuffix="_newOnly",
    df = df_UK_NHS_daily_COVID_deaths,
    sumAges=True
)#.iplot(line=dict(color='firebrick', width=4))


a = plot(tmp.values, tmp.index, now=False)
a[0]["marker"]=dict(color="black", size=8)
a[0]["mode"]="markers"
for ind in range(1,len(a)):
    a[ind]["opacity"]=1./np.sqrt(len(a)/2.)
plt(go.Figure(data=a, layout=go.Layout(title="Daily in-hospital total COVID deaths in the UK")))

In [ ]:
# Plot new hospitalisations per day
tmp = joinSimulations(
    loadDir = loadDir,
    paramTable=paramTable_goodOnly,
    simRowIndices = paramTable_goodOnly.sort_values("likelihood_total").tail(200).index,
    paramTable_startTimeColname = "realStartDate",
    likFunc = likFunc_newHospPositive,
    simSuffix="_newOnly",
    df = df_CHESS_newCOVID,
    sumAges=True
)

a = plot(tmp.values, tmp.index, now=False)
a[0]["marker"]=dict(color="black", size=8)
a[0]["mode"]="markers"
for ind in range(1,len(a)):
    a[ind]["opacity"]=1./np.sqrt(len(a)/2.)
plt(go.Figure(data=a, layout=go.Layout(title="Daily in-hospital positive tests in the UK")))

In [ ]:
# Plot new infections per day
tmp = joinSimulations(
    loadDir = loadDir,
    paramTable=paramTable_goodOnly,
    simRowIndices = paramTable_goodOnly.sort_values("likelihood_total").tail(200).index,
    paramTable_startTimeColname = "realStartDate",
    likFunc = functools.partial(
        likFunc_dummyForPlotting, 
        projFunc= lambda sim: sim[:,1].sum(2).sum(1)
        ),
    simSuffix="_newOnly",
    df = None,
    sumAges=True
)

a = plot(tmp.values, tmp.index, now=False)
for ind in range(len(a)):
    a[ind]["opacity"]=1./np.sqrt(len(a)/2.)

plt(go.Figure(data=a, layout=go.Layout(title="Daily new infections in the UK [no data available]")))